In [ ]:
%pip install -qqq llama-index llama-hub langchain accelerate==0.21.0 bitsandbytes==0.40.2 transformers sentence-transformers
%pip install llama-index-llms-huggingface
%pip install --upgrade openai
%pip install llama-index-embeddings-langchain
%pip install pydrive2
%pip install dropbox
%pip install boto3
#%pip install ctransformers[cuda]
%pip install llama-index-llms-llama-cpp
%pip install matplotlib
%pip install chainlit

In [17]:
# Libraries

# %load_ext autoreload
# %autoreload 2

import nest_asyncio
nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import json
import torch
from pathlib import Path
import pandas as pd
pd.set_option("display.max_colwidth", None)

from copy import deepcopy

# transformers
from transformers import BitsAndBytesConfig

# llama_index
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import SimpleDirectoryReader, download_loader, Document, VectorStoreIndex, ServiceContext
from llama_index.core.node_parser import SentenceSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core.response.notebook_utils import display_source_node
from IPython.display import Markdown, display, HTML
from llama_index.core.retrievers import VectorIndexRetriever

import os
import chainlit as cl
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import CTransformers
from langchain_community.embeddings import HuggingFaceEmbeddings



DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

import io
import os
import tempfile
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaIoBaseDownload
import dropbox
from dropbox.exceptions import AuthError, ApiError
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import tempfile


In [ ]:
#data_pipeline code

#We can specify some directory locally to create SimpleDirectoryReader for example

documents = SimpleDirectoryReader("path_to_folder", filename_as_id=True).load_data()    

#or we can get data from Cloud, I have prepared 3 code for 3 different sources(Google drive, AWS S3 Storage, and Dropbox)

def download_files_from_google_drive(credentials_file, folder_id):
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(credentials_file)
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    query = f"'{folder_id}' in parents and trashed=false"
    file_list = drive.ListFile({'q': query}).GetList()
    
    for file in file_list:
        print(f'Downloading {file["title"]} from Google Drive...')
        temp_file_path = os.path.join(temp_dir, file['title'])
        file_io = io.FileIO(temp_file_path, 'wb')
        request = drive.auth.service.files().get_media(fileId=file['id'])
        downloader = MediaIoBaseDownload(file_io, request)
        done = False
        while not done:
            _, done = downloader.next_chunk()
        file_io.close()

    print("Completed downloading all files from Google Drive.")

def download_files_from_s3(region, access_key, secret_access_key, bucket_name):
    s3 = boto3.resource(
        's3',
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_access_key,
        region_name=region
    )
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.all():
        file_path = os.path.join(temp_dir, obj.key)
        bucket.download_file(obj.key, file_path)
        print(f"Downloaded {obj.key} from AWS S3 to {file_path}")

def download_files_from_dropbox(access_token, path=""):
    try:
        dbx = dropbox.Dropbox(access_token)
        files = dbx.files_list_folder(path).entries
        for file_metadata in files:
            if isinstance(file_metadata, dropbox.files.FileMetadata):
                file_path = file_metadata.path_lower
                temp_file_path = os.path.join(temp_dir, os.path.basename(file_path))
                _, result = dbx.files_download(file_path)
                with open(temp_file_path, 'wb') as f:
                    f.write(result.content)
                print(f"Downloaded {file_path} from Dropbox to {temp_file_path}")
    except AuthError as e:
        print('Error downloading from Dropbox: ' + str(e))
    except ApiError as e:
        print('API error: ' + str(e))

# Google Drive
google_credential_file = "-" #google cloud credentials
google_folder_id = "-" #Folder Id to specific folder

# Dropbox
dropbox_access_token = "-" #dropbox api token

# AWS S3 credientials and ids
aws_region = "-"
aws_access_key_id = "-"
aws_secret_access_key ="-"
aws_bucket_name = "-"

with tempfile.TemporaryDirectory() as temp_dir:
    
    #download_files_from_google_drive(google_credential_file, google_folder_id)    

    #download_files_from_dropbox(dropbox_access_token)    

    #download_files_from_s3(aws_region, aws_access_key_id, aws_secret_access_key, aws_bucket_name)

    documents = SimpleDirectoryReader(temp_dir, filename_as_id=True).load_data()    

    print(f"All files are temporarily stored in {temp_dir}")
    


In [ ]:



#Rag code 
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = """
        You are a business chatbot designed to assist users with their business-related tasks. Use the information from the DOCUMENTS section to provide accurate answers but act as if you knew this information innately.

        DOCUMENTS:
        {documents}

        Please answer questions based solely on the data I have provided. If the information is not present in the documents, please respond with 'I don't have enough information to answer that.' Avoid making assumptions or providing speculative answers. Please adhere strictly to the data provided.

        When answering questions, you should:
        
        Rely solely on the information present in the documents.
        Avoid using external knowledge or making assumptions.
        Respond with 'I don't have enough information to answer that.' if the data is not in the documents.
        Provide clear, non-speculative answers.
        Aim to answer to the best of your ability based on the data provided.
        If unsure, simply state, "I don't know." For any queries outside of the DOCUMENTs section, your response should be "I do not know about this."

        Your primary objectives are:

        Answer Questions about the Provided Documents:
          - Your role is to provide accurate and relevant information in response to user queries about the documents provided.
          - If a user asks a question pertaining to a specific detail in the documents, your task is to extract the relevant information and provide a clear answer.
          - For example, if a user asks about the ownership structure or financial status of the company mentioned in the documents, you should respond with precise details.

        """

  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt



llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/zephyr-7B-alpha-GGUF/resolve/main/zephyr-7b-alpha.Q5_K_M.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=500,
    context_window=3900,
    generate_kwargs={ "top_k": 50, "top_p": 0.95},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    verbose=True,

) 
 

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large")

In [ ]:
# Using the LlamaDebugHandler to print the trace of the sub questions
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

# ServiceContext
service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=embed_model,
                                               callback_manager=callback_manager,
                                               chunk_size_limit=512
                                               )

In [ ]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    index = load_index_from_storage(storage_context, index_id="vector_index")
except:
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    index.storage_context.persist("./storage")
#print('persisting to disk')

In [ ]:
query_input = "who is george bush?" #Anything unrelated
response = index.as_query_engine().query(query_input)